# SQL 的五十道練習：初學者友善的資料庫八週專班

> 第三天

[數聚點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/kaggleSurvey2022.db" AS kaggleSurvey2022;

## 子查詢

## 子查詢

如果是一段 SQL 敘述中包括另外一段 SQL 敘述、先後使用多個 SQL 敘述，先執行的 SQL 敘述查詢結果將會成回後執行的 SQL 敘述中的依據，這樣的 SQL 敘述結構就稱為子查詢（Subquery）。

## 複合函數（Composite functions）

`SUBSTR('Boston', 1, 3)` 函數的輸出為 `'Bos'`，成為 `UPPER()` 函數的輸入，最後的輸出為 `'BOS'`。

In [4]:
SELECT 'Boston' AS city,
       SUBSTR('Boston', 1, 3),
       UPPER(SUBSTR('Boston', 1, 3)) AS composite_function;

city,"SUBSTR('Boston', 1, 3)",composite_function
Boston,Bos,BOS


## 常見的子查詢結構外型

接續在 `WHERE` 保留字後的結構外型。

```sql
SELECT columns
  FROM table
 WHERE (SELECT columns FROM table ...);
```

## 常見的子查詢結構外型（續）

接續在 `SELECT` 保留字後的結構外型。

```sql
SELECT (SELECT columns FROM table ...)
  FROM table;
```

## 常見的子查詢結構外型（續）

接續在 `FROM` 保留字後的結構外型，這裡要注意的是先前別名是針對欄位名稱，這裡則是將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）的存在。

```sql
SELECT columns
  FROM (SELECT columns FROM table ...) AS alias;
```

## 常見的子查詢應用情境

1. 接續在 `WHERE` 保留字後的結構外型，應用於篩選資料表觀測值的條件必須要先經過一個 SQL 敘述查詢才能夠建立。
2. 接續在 `SELECT` 保留字後的結構外型，應用於衍生計算欄位的算式部分必須要先經過一個 SQL 敘述查詢才能夠獲得。
3. 接續在 `FROM` 保留字後的結構外型，應用於將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）對待來取得所需資訊。

## 情境一：接續在 `WHERE` 保留字後的結構外型

將問題 "What programming languages do you use on a regular basis? (Select all that apply)" 的所有回覆篩選出來。

```sql
SELECT *
  FROM responses
 WHERE question_id = x;
 
SELECT id
  FROM questions
 WHERE description LIKE '%programming language%';
```

## 將 `x` 替換為 SQL 敘述，並用小括號 `()` 包裝起來

就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

```sql
SELECT *
  FROM responses
 WHERE question_id = (
                         SELECT id
                           FROM questions
                          WHERE description LIKE '%programming language%'
                     );
```

## 值得注意的地方

- 子查詢的結構外型也只能有一個分號 `;` 來標註 SQL 敘述的結束，因此替換之後要記得只留下最後執行 SQL 敘述的分號。
- 替換之後因為排版變得比較亂，這時可以善用 SQLiteStudio 的 Format SQL 功能讓寫作的 SQL 敘述之編排、格式和設計具備更高的可讀性。

## 情境二：接續在 `SELECT` 保留字後的結構外型

將問題 "What programming languages do you use on a regular basis? (Select all that apply)" 的回覆計算為比例呈現。

```sql
SELECT choice,
       COUNT(*)*1.0 / x AS n_responses
  FROM responses
 WHERE question_id = 11
 GROUP BY choice
 ORDER BY n_responses DESC;
 
SELECT COUNT(*) AS total_responses
  FROM responses
 WHERE question_id = 11;
```

## 將 `x` 替換為 SQL 敘述，並用小括號 `()` 包裝起來

就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

```sql
SELECT choice,
       COUNT( * )*1.0 / (
                        SELECT COUNT( * ) AS total_responses
                          FROM responses
                         WHERE question_id = 11
                    )
       AS n_responses
  FROM responses
 WHERE question_id = 11
 GROUP BY choice
 ORDER BY n_responses DESC;
```

## 情境三：接續在 `FROM` 保留字後的結構外型

將 `nba` 資料庫平均身高大於等於 2 公尺的鋒衛位置挑選出來。

```sql
SELECT pos,
       AVG(heightMeters) AS avg_height_meters
  FROM players
 GROUP BY pos
HAVING avg_height_meters >= 2;
```

## 情境三：接續在 FROM 保留字後的結構外型（續）

```sql
SELECT *
  FROM x
 WHERE avg_height_meters >= 2;

SELECT pos,
       AVG(heightMeters) AS avg_height_meters
  FROM players
 GROUP BY pos;
```

## 將 `x` 替換為 SQL 敘述，並用小括號 `()` 包裝起來

就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [5]:
SELECT *
  FROM (
           SELECT pos,
                  AVG(heightMeters) AS avg_height_meters
             FROM players
            GROUP BY pos
       )
       AS avg_height_meters_by_pos
 WHERE avg_height_meters >= 2;

pos,avg_height_meters
C,2.11878787878788
C-F,2.095
F,2.02348484848485
F-C,2.0785
F-G,2.00217391304348


## 同學上一次直播問的問題

> 請問如果資料只有4個，OFFSET 5會顯示資料嗎？

In [6]:
SELECT *
  FROM (
           SELECT *
             FROM movies
            LIMIT 4
       )
       AS four_rows
 LIMIT 4 OFFSET 5;

Empty set (0.00 sec)

## 兩次以上的子查詢結構

我們想知道 `imdb` 資料庫哪幾部電影 Tom Hanks 有演出？

```sql
SELECT title
  FROM movies
 WHERE id IN x;

SELECT movie_id
  FROM casting
 WHERE actor_id = y;

SELECT id
  FROM actors
 WHERE name = 'Tom Hanks';
```

## 將 `x` 與 `y` 依序替換為 SQL 敘述，並用小括號 `()` 包裝起來

就能成功將本來應該分三次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [7]:
SELECT title
  FROM movies
 WHERE id IN (
           SELECT movie_id
             FROM casting
            WHERE actor_id = (
                                 SELECT id
                                   FROM actors
                                  WHERE name = 'Tom Hanks'
                             )
       );

title
Forrest Gump
Saving Private Ryan
The Green Mile
Toy Story
Toy Story 3
Catch Me If You Can


## 垂直與水平合併

## 關聯

所謂的關聯，具體來說就是讓資料從兩個維度合併：

1. 垂直合併：從垂直的方向關聯資料的列（觀測值）。
2. 水平合併：從水平的方向關聯資料的欄（變數）。

更簡單的說明，垂直合併就是列的結合，`(m, n)` 外型與 `(m, n)` 外型的資料垂直合併後為 `(2m, n)` 外型；水平合併就是欄的結合，`(m, n)` 外型與 `(m, n)` 外型的資料水平合併後為 `(m, 2n)` 外型（在觀測值一對一對應的前提之下）。

## 以 `JOIN` 水平合併

- 資料是以水平維度進行合併，因此還會有左資料表與右資料表之分。
- 在 `FROM` 保留字之後的資料表被稱為左資料表（Left table）或稱主要資料表；在 `JOIN` 保留字之後的資料表被稱為右資料表（Right table）或稱次要資料表。

```sql
SELECT left_table.columns,
       right_table.columns
  FROM left_table
  JOIN right_table
    ON left_table.join_key = right_table.join_key;
```

## 添加 `JOIN` 與 `ON` 納入更多的次要資料表

- 在 `ON` 保留字之後的敘述包含了資料表名稱與連接鍵的宣告 `table.join_key` 中間用 `.` 來區分資料表名稱與連接鍵。
- 而在 `SELECT` 保留字之後所選擇的欄位，因為能夠從主要資料表與次要資料表中選擇，為了提升可讀性、同時也避免模糊錯誤（Ambiguity error）的發生，也同樣會以 `table.columns` 明確表示資料表名稱與所選擇欄位。

```sql
SELECT left_table.columns,
       right_table_one.columns,
       right_table_two.columns
  FROM left_table
  JOIN right_table_one
    ON left_table.join_key = right_table_one.join_key
  JOIN right_table_two
    ON left_table.join_key|right_table_one.join_key = right_table_two.join_key;
```

## 用水平合併來回答這個問題

- 我們想知道 `imdb` 資料庫哪幾部電影 Tom Hanks 有演出？
    - 先知道 Tom Hanks 的演員編號 `actor_id` 是多少。
    - 從 `casting` 資料表對應出哪些電影編號 `movie_id`。
    - 進而再由 `movies` 資料表取得電影資訊 `title`。

In [8]:
SELECT movies.title
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE actors.name = 'Tom Hanks';

title
Forrest Gump
Saving Private Ryan
The Green Mile
Toy Story
Toy Story 3
Catch Me If You Can


## 什麼是實體關係圖（Entity Relationship Diagram, ER-Diagram）

- 實體關係圖會將關聯式資料庫中每個資料表像清單般展開，最上方是該資料表名稱。
- 置頂且粗體的欄位名稱則標註了該資料表中用來區隔「不重複」觀測值的變數，也就是所謂的主鍵（Primary Key），主鍵被用來確認資料的獨一性。
- 資料表與資料表之間的連線則描述兩者能夠透過連接鍵關聯，連接鍵也稱為外鍵（Foreign Key），外鍵被用來確認資料的參考完整性。
- 我們可以使用 DBeaver 生成學習資料庫的實體關係圖。

## 實體（Entity）、屬性（Attribute）與鍵（Key）等其實只是「換句話說」

- 關聯（Relation）、實體（Entity）：資料表（Table）
- 元組（Tuple）：列（Row）、觀測值（Observation）或記錄（Record）
- 屬性（Attribute）：欄（Column）、變數（Variable）、鍵（Key）或欄位（Field）

## `covid19` 資料庫的實體關係圖

可以得知 `daily_report` 的主鍵為 `Combined_Key`、可以透過 `daily_report.Combined_Key` 與 `lookup_table.Combined_Key` 連接；`lookup_table` 主鍵為 `UID`、可以透過 `lookup_table.Country_Region` 與 `time_series.Country_Region` 連接。

![](../images/covid19-erd.png)

## `nba` 資料庫的實體關係圖

可以得知 `career_summaries` 的主鍵為 `personId`、可以透過 `career_summaries.personId` 與 `players.personId` 連接；`teams` 主鍵為 `teamId`、可以透過 `teams.teamId` 與 `players.teamId` 連接。

![](../images/nba-erd.png)

## `twElection2020` 資料庫的實體關係圖

可以得知 `admin_regions` 的主鍵為 `id`、可以透過 `admin_regions.id` 與 `legislative_at_large.admin_region_id`、`legislative_regional.admin_region_id`、`presidential.admin_region_id` 連接；`candidates` 主鍵為 `id`、可以透過 `candidates.id` 與 `presidential.candidate_id`、`legislative_regional.candidate_id` 連接，透過 `candidates.party_id` 與 `parties.id` 連接；`parties` 主鍵為 `id`、可以透過 `parties.id` 與 `legislative_at_large.party_id`、`candidates.party_id` 連接。

![](../images/twElection2020-erd.png)

## `twElection2022` 資料庫的實體關係圖

可以得知 `admin_regions` 的主鍵為 `id`、可以透過 `admin_regions.id` 與 `councillors.admin_region_id`、`mayors.admin_region_id` 連接；`candidates` 主鍵為 `id`、可以透過 `candidates.id` 與 `councillors.candidate_id`、`mayors.candidate_id` 連接，透過 `candidates.party_id` 與 `parties.id` 連接；`parties` 主鍵為 `id`、可以透過 `parties.id` 與 `councillors.party_id`、`mayors.party_id` 連接。

![](../images/twElection2022-erd.png)

## `kaggleSurvey2022` 資料庫的實體關係圖

可以得知 `questions` 的主鍵為 `id`、可以透過 `questions.id` 與 `responses.question_id` 連接。

![](../images/kaggleSurvey2022-erd.png)

## 理解實體關係圖、主鍵、外鍵等觀念的最佳方式

- 自己建立學習資料庫：
    - 第一階段：使用清理後的 `.csv` 檔案建立學習資料庫。
    - 第二階段：使用一個程式語言整理資料之後建立學習資料庫。
- 在第四天的直播中示範。

## 水平合併的效果

- 將三個資料表的欄都集中到了一個查詢結果中：
    - `id:runtime` 來自 `movies` 資料表。
    - `movie_id:ord` 來自 `casting` 資料表。
    - `id:name` 來自 `actors` 資料表。

In [9]:
SELECT *
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE actors.name = 'Tom Hanks';

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord,id,name
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2865,1,2865,Tom Hanks
26,Saving Private Ryan,1998,8.6,Steven Spielberg,169,26,2865,1,2865,Tom Hanks
27,The Green Mile,1999,8.6,Frank Darabont,189,27,2865,1,2865,Tom Hanks
82,Toy Story,1995,8.3,John Lasseter,81,82,2865,1,2865,Tom Hanks
112,Toy Story 3,2010,8.2,Lee Unkrich,103,112,2865,1,2865,Tom Hanks
189,Catch Me If You Can,2002,8.1,Steven Spielberg,141,189,2865,2,2865,Tom Hanks


## 值得注意的水平合併特性

- 留意資料的映射關係。
- 留意觀測值的交集、對稱差集與聯集關係。
    - 內部連接（交集）。
    - 左外部連接（左資料表差集右資料表）。
    - 右外部連接（右資料表差集左資料表）。
    - 全外部連接（聯集）。

## 資料的映射關係

一部電影對應多個演員。

In [10]:
SELECT movies.title,
       casting.actor_id
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
 WHERE movies.id = 1;

title,actor_id
The Shawshank Redemption,2853
The Shawshank Redemption,2097
The Shawshank Redemption,333
The Shawshank Redemption,3029
The Shawshank Redemption,533
The Shawshank Redemption,1042
The Shawshank Redemption,1923
The Shawshank Redemption,1319
The Shawshank Redemption,1370
The Shawshank Redemption,1724


## 資料的映射關係（續）

一個演員對應多部電影。

In [11]:
SELECT actors.name,
       casting.movie_id
  FROM actors
  JOIN casting
    ON actors.id = casting.actor_id
 WHERE actors.name = 'Tom Hanks';

name,movie_id
Tom Hanks,12
Tom Hanks,26
Tom Hanks,27
Tom Hanks,82
Tom Hanks,112
Tom Hanks,189


## 留意觀測值的的交集、對稱差集與聯集關係

舉例來說，如果我們利用子查詢設計左表格為阿甘正傳與搶救雷恩大兵的電影資料；設計右表格為阿甘正傳與神鬼交鋒的演員卡司。

In [12]:
SELECT *
  FROM movies
 WHERE id IN (12, 26); -- AS left_table

id,title,release_year,rating,director,runtime
12,Forrest Gump,1994,8.8,Robert Zemeckis,142
26,Saving Private Ryan,1998,8.6,Steven Spielberg,169


In [13]:
SELECT *
  FROM casting
 WHERE movie_id IN (12, 189); -- AS right_table

movie_id,actor_id,ord
12,2865,1
12,2404,2
12,2571,3
12,2017,4
12,1126,5
12,1007,6
12,337,7
12,1488,8
12,2575,9
12,1894,10


## `JOIN` 觀測值的的交集、內部連接

將左表格（阿甘正傳與搶救雷恩大兵的電影資料）與右表格（阿甘正傳與神鬼交鋒的演員卡司）水平合併，結果會是左表格與右表格**交集**的觀測值，也就是阿甘正傳的電影資料、演員卡司，而搶救雷恩大兵的電影資料與神鬼交鋒的演員卡司都不會出現在查詢結果中。

In [14]:
SELECT *
  FROM (
           SELECT *
             FROM movies
            WHERE id IN (12, 26)
       )
       AS left_table
  JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (12, 189)
       )
       AS right_table
    ON left_table.id = right_table.movie_id;

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2865,1
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2404,2
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2571,3
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2017,4
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1126,5
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1007,6
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,337,7
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1488,8
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2575,9
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1894,10


## `LEFT JOIN` 觀測值的差集（左資料表差集右資料表）、左外部連接

改使用 `LEFT JOIN` 可以變為保留左表格（主要表格）的觀測值。也就是阿甘正傳與搶救雷恩大兵的電影資料、阿甘正傳的演員卡司，搶救雷恩大兵因為沒有演員卡司可以連接，所以得到空值 `NULL`。

In [15]:
SELECT *
  FROM (
           SELECT *
             FROM movies
            WHERE id IN (12, 26)
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (12, 189)
       )
       AS right_table
    ON left_table.id = right_table.movie_id;

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2865,1
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2404,2
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2571,3
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2017,4
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1126,5
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1007,6
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,337,7
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1488,8
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,2575,9
12,Forrest Gump,1994,8.8,Robert Zemeckis,142,12,1894,10


## `RIGHT JOIN` 觀測值的差集（右資料表差集左資料表）、右外部連接

雖然 SQLite 關聯式資料庫管理系統並沒有 `RIGHT JOIN` 可以直接使用來變為保留右表格（次要表格）的觀測值，但因為左、右是相對的，其實可以沿用 `LEFT JOIN` 但是把左表格與右表格互換，藉此達到預期的效果。也就是阿甘正傳與神鬼交鋒的演員卡司、阿甘正傳的電影資料，神鬼交鋒因為沒有電影資料可以連接，所以得到空值 `NULL`。

In [16]:
SELECT *
  FROM (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175)  
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan')
       )
       AS right_table
    ON left_table.movie_id = right_table.id;

movie_id,actor_id,ord,id,title,release_year,rating,director,runtime
11,782,1,NULL,NULL,NULL,NULL,NULL,NULL
11,352,2,NULL,NULL,NULL,NULL,NULL,NULL
11,1989,3,NULL,NULL,NULL,NULL,NULL,NULL
11,3085,4,NULL,NULL,NULL,NULL,NULL,NULL
11,2448,5,NULL,NULL,NULL,NULL,NULL,NULL
11,638,6,NULL,NULL,NULL,NULL,NULL,NULL
11,1013,7,NULL,NULL,NULL,NULL,NULL,NULL
11,867,8,NULL,NULL,NULL,NULL,NULL,NULL
11,1165,9,NULL,NULL,NULL,NULL,NULL,NULL
11,507,10,NULL,NULL,NULL,NULL,NULL,NULL


## `FULL JOIN` 觀測值的聯集、全外部連接

雖然 SQLite 關聯式資料庫管理系統並沒有 `FULL JOIN` 可以直接使用來變為保留左、右表格（主、次要表格）的觀測值，但我們可以把前述兩個 `LEFT JOIN` 的結果透過 `UNION` 垂直合併，藉此達到預期的效果。也就是阿甘正傳的電影資料與演員卡司、搶救雷恩大兵的電影資料、神鬼交鋒的演員卡司，神鬼交鋒因為沒有電影資料可以連接、搶救雷恩大兵因為沒有演員卡司可以連接，所以都得到空值 `NULL`。

In [17]:
SELECT left_table.id,
       left_table.title,
       right_table.actor_id
  FROM (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan') 
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175) 
       )
       AS right_table
    ON left_table.id = right_table.movie_id
 UNION
SELECT right_table.id,
       right_table.title,
       left_table.actor_id
  FROM (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175)  
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan')
       )
       AS right_table
    ON left_table.movie_id = right_table.id

id,title,actor_id
NULL,NULL,352
NULL,NULL,369
NULL,NULL,506
NULL,NULL,507
NULL,NULL,630
NULL,NULL,638
NULL,NULL,780
NULL,NULL,782
NULL,NULL,867
NULL,NULL,880


## 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定

```sql
 SELECT DISTINCT columns AS alias,
        CASE WHEN condition_1 THEN result_1
             WHEN condition_2 THEN result_2
             ...
             ELSE result_n END AS alias
   FROM left_table
   JOIN | LEFT JOIN right_table
     ON left_table.join_key = right_table.join_key
  WHERE conditions
  GROUP BY columns
 HAVING conditions
  UNION | UNION ALL
Another SQL statement
  ORDER BY columns DESC
  LIMIT m OFFSET m;
```

## 第三天的額外練習題

- 使用 `kaggleSurvey2022.db` 與 `twElection2022.db`
- [練習題連結](https://gke.mybinder.org/v2/gh/datainpoint/environment-hahow-sqlfifty/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fdatainpoint%252Fclassroom-hahow-sqlfifty%26urlpath%3Dtree%252Fclassroom-hahow-sqlfifty%252Fworkshop%252Fexercises_day_three.ipynb%26branch%3Dmain)

## 學長姐常見問題彙整

> 當需要的資料來自不同資料表(情境三)如何判斷何時要使用子查詢，何時使用join的方式來達到目的會比較有效率呢？

## 學長姐常見問題彙整

> 這章作業有很多要確認的欄位，都要土法煉鋼叫出來看，SELECT  *  FROM table_name Limit 1; 查看所有欄位,有沒有比較方便的方式可以代替呢?

## 學長姐常見問題彙整

> 我是程式語言的新手，想要請教一下，因為覺得實務上join表格的狀況，可能會很常出現。因為在上這堂課的時候，發現如果今天是一個實際的問題時，我會有兩個困擾：1.可能table參數不夠熟悉，所以都需要把該資料庫所有的table先select出來，然後再去思考怎麼join 2.能有快速的方法可以得知各table的屬性嗎?那想請教老師除了多做練習，有什麼tip或思考邏輯是可以知道表格間怎麼join會能得到要的答案(尤其是水平合併)?

## 學長姐常見問題彙整

> 老師您好，我想請問關於左、右表格的書寫順序是只要照著自己的思考方式書寫就可以了嗎?如果未來碰到右表格不只一個的狀況，不同的書寫順序會不會有什麼問題?

## 成為資料分析師的學習地圖

- [How I Would Learn Data Analysis (If I Could Start Over)](https://youtu.be/qWEHO8b6WbA)
- [How I would learn Machine Learning (if I could start over)](https://youtu.be/wtolixa9XTg)

## 選定一個作品集的平台

- [GitHub](https://github.com)
- [HackMD](https://hackmd.io)
- 部落格：英文選擇 [Medium](https://medium.com)、中文可以選擇 [方格子](https://vocus.cc)
- 自建部落格：[Hugo](https://gohugo.io)、[Gatsby](https://www.gatsbyjs.com)

## 選定寫作工具

- 假使選擇了「部落格」以外的平台，建議**一定**要學習標記語言（Markdown Language）。
- [Markdown 文件](https://markdown.tw)
    - [Jupyter Notebook](https://jupyter.org)
    - [Quarto](https://quarto.org/)

## 第零站：跟電腦變成好朋友

寫程式是理解資料科學、統計知識與機器學習理論不可或缺的工具與手段，因此學習的起點必須從程式設計起步，而在開始寫程式之前我們應該跟電腦變成好朋友！首先學習操作終端機以及純文字編輯器，行有餘力的話，再多認識 Git 和 GitHub。

## 第一站：使用 SQL 查詢資料

從 2022 年的 Kaggle 調查我們得知資料分析師使用最多的語言依序是 Python -> SQL -> R，我會推薦從 SQL 開始，原因是 SQL 作為由資料庫中查詢資料的語言，與人類語言的相似度高、語法單純並且是即戰力的工具。

## 第二站：使用商業智能軟體作探索性分析

懂得如何寫作 SQL 之後，可以跟試算表、商業智能軟體搭配開始進行探索性分析，從 2022 年的 Kaggle 調查我們得知 Tableau 與 Microsoft Power BI 是現在最普遍的商業智能軟體，而這兩個都有免費的版本可供個人電腦使用，如果是使用 Windows 作業系統的學員可以都試用看看，使用 macOS 的學員就試用 Tableau。

## 第三站：學習 Python 程式設計

想要勝任工程導向的資料科學團隊職缺，必須學會一個在專案的各個應用都能介接、並且能夠自己整併清理資料的程式語言，從 2022 年的 Kaggle 調查我們得知資料分析師的首選是 Python 程式語言。

## 第四站：機器學習應用

先使用機器學習模組、套件實作 Kaggle 專案，至於為何所引用的函數與類別能夠完成機器學習應用，則留待後續在研讀理論後融會貫通。

## 第五站：學習 R 程式設計

機器學習理論會牽涉到程式設計與統計理論，特別是條件機率與機率分配等部分，學習統計理論最好的程式語言是 R 語言，如果已經掌握 Python 程式設計以及資料科學模組 NumPy、Pandas，在 R 語言的學習進展將會非常快速，我們會發現兩者在面對資料的處理邏輯上其實有異曲同工之妙。

## 第六站：統計與機器學習理論

不論是找相關工作的技術面試或者實務應用，除了能夠使用現成的機器學習模組、套件，也要能夠理解相關函數、類別的參數設定以及其背後的理論對應，才能夠順利通過面試或勝任工作。